<a href="https://colab.research.google.com/github/LucieEngel/Pneumonia-classifier/blob/main/Pnemonia_classifier_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pneumonia - classifier from chest X-rays

Import packages

In [1]:
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

## Step 1 : Load data

In [3]:
# Connect to my drive 
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
# Change current directory 
os.chdir("drive/My Drive/Colab Notebooks")

In [6]:
# Read dataset summary
pd.read_csv('archive/Chest_xray_Corona_dataset_Summary.csv', index_col=0)

,Label,Label_1_Virus_category,Label_2_Virus_category,Image_Count
0,Normal,NaN,NaN,1576
1,Pnemonia,Stress-Smoking,ARDS,2
2,Pnemonia,Virus,NaN,1493
3,Pnemonia,Virus,COVID-19,58
4,Pnemonia,Virus,SARS,4
5,Pnemonia,bacteria,NaN,2772
6,Pnemonia,bacteria,Streptococcus,5


In [7]:
# Read meta data 
meta_data = pd.read_csv('archive/Chest_xray_Corona_Metadata.csv', index_col=0)
meta_data

,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN
...,...,...,...,...,...
5928,person1637_virus_2834.jpeg,Pnemonia,TEST,NaN,Virus
5929,person1635_virus_2831.jpeg,Pnemonia,TEST,NaN,Virus
5930,person1634_virus_2830.jpeg,Pnemonia,TEST,NaN,Virus
5931,person1633_virus_2829.jpeg,Pnemonia,TEST,NaN,Virus


In [9]:
len(meta_data[(meta_data.Dataset_type == 'TRAIN') & (meta_data.Label == 'Pnemonia')])/len(meta_data[(meta_data.Dataset_type == 'TRAIN')])

0.7461218312523648

In [10]:
meta_data.Dataset_type.value_counts()

TRAIN    5286
TEST      624
Name: Dataset_type, dtype: int64

In [11]:
# Define training labels 

traindf = meta_data.loc[meta_data.Dataset_type=="TRAIN"]
traindf.head()

,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN


In [12]:
# Define testing labels 

testdf = meta_data.loc[meta_data.Dataset_type=="TEST"]
testdf.head()

,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
5309,IM-0021-0001.jpeg,Normal,TEST,NaN,NaN
5310,IM-0019-0001.jpeg,Normal,TEST,NaN,NaN
5311,IM-0017-0001.jpeg,Normal,TEST,NaN,NaN
5312,IM-0016-0001.jpeg,Normal,TEST,NaN,NaN
5313,IM-0015-0001.jpeg,Normal,TEST,NaN,NaN


In [13]:
from keras_preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale = 1./255., validation_split = 0.1)

In [15]:
# Train generator

train_generator=datagen.flow_from_dataframe(
    dataframe = traindf,
    directory = 'archive/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train',
    x_col = 'X_ray_image_name',
    y_col = 'Label',
    subset = "training",
    batch_size = 32,
    seed = 42, 
    shuffle = True,
    classes=['Normal', 'Pnemonia'],
    class_mode='binary',
    target_size = (128, 128))

valid_generator=datagen.flow_from_dataframe(
    dataframe = traindf,
    directory = 'archive/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train',
    x_col = 'X_ray_image_name',
    y_col = 'Label',
    subset = "validation",
    batch_size = 32,
    seed = 42, 
    shuffle = True,
    classes=['Normal', 'Pnemonia'],
    class_mode='binary',
    target_size = (128, 128))

test_datagen = ImageDataGenerator(rescale = 1./255.)

test_generator=test_datagen.flow_from_dataframe(
    dataframe = testdf,
    directory = 'archive/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test',
    x_col = 'X_ray_image_name',
    y_col = 'Label',
    batch_size = 32,
    seed = 42, 
    shuffle = True,
    classes=['Normal', 'Pnemonia'],
    class_mode='binary',
    target_size = (128, 128))

Found 4758 validated image filenames belonging to 2 classes.
Found 528 validated image filenames belonging to 2 classes.
Found 624 validated image filenames belonging to 2 classes.


In [16]:
train_generator.class_indices

{'Normal': 0, 'Pnemonia': 1}

In [ ]:
#@title
# #@title
# # Reproducability
# def set_seed(seed=31415):
#     np.random.seed(seed)
#     tf.random.set_seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     os.environ['TF_DETERMINISTIC_OPS'] = '1'
# set_seed()

# # Set Matplotlib defaults
# plt.rc('figure', autolayout=True)
# plt.rc('axes', labelweight='bold', labelsize='large',
#        titleweight='bold', titlesize=18, titlepad=10)
# plt.rc('image', cmap='magma')
# warnings.filterwarnings("ignore") # to clean up output cells


# # Load training and validation sets
# # ds_train_ = image_dataset_from_directory(
# #     'archive/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train',
# #     labels='inferred',
# #     label_mode='binary',
# #     image_size=[128, 128],
# #     interpolation='nearest',
# #     validation_split = 0.3,
# #     subset = "training"
# #     batch_size=64,
# #     shuffle=True,
# # )

# # ds_valid_ = image_dataset_from_directory(
# #     'archive/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train',
# #     labels='inferred',
# #     label_mode='binary',
# #     image_size=[128, 128],
# #     interpolation='nearest',
# #     validation_split = 0.3,
# #     subset = "validation"
# #     batch_size=64,
# #     shuffle=True,
# # )

# # ds_test_ = image_dataset_from_directory(
# #     'archive/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test',
# #     labels='inferred',
# #     label_mode='binary',
# #     image_size=[128, 128],
# #     interpolation='nearest',
# #     batch_size=64,
# #     shuffle=False,
# # )






# # Data Pipeline
# def convert_to_float(image, label):
#     image = tf.image.convert_image_dtype(image, dtype=tf.float32)
#     return image, label

# AUTOTUNE = tf.data.experimental.AUTOTUNE

# ds_train = (
#     ds_train_
#     .map(convert_to_float)
#     .cache()
#     .prefetch(buffer_size=AUTOTUNE)
# )
# ds_valid_ = (
#     ds_valid_
#     .map(convert_to_float)
#     .cache()
#     .prefetch(buffer_size=AUTOTUNE)
# )

##Step 2 : Define Model 

In [17]:
from tensorflow import keras
from tensorflow.keras import layers

Use transfer learning 

In [18]:
input_t = tf.keras.Input(shape=(128,128,3))
pretrained_base = tf.keras.applications.ResNet50(include_top = False, weights = 'imagenet', input_tensor= input_t)

94781440/94765736 [==============================] - 1s 0us/step


In [19]:
pretrained_base.trainable = True

In [20]:
model = keras.Sequential([
    pretrained_base,
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

In [21]:
for i, layer in enumerate(model.layers):
  print(i,layer.name,"-",layer.trainable)

0 resnet50 - True
1 flatten - True
2 dense - True
3 dense_1 - True
4 dense_2 - True


In [ ]:
# model = keras.Sequential([
#     # Block One
#     layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same',
#                   input_shape=[128, 128, 3]),
#     layers.MaxPool2D(),

#     # Block Two
#     layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
#     layers.MaxPool2D(),

#     # Block Three
#     layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
#     layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
#     layers.MaxPool2D(),
    

#     # Head
#     layers.Flatten(),
#     layers.Dense(6, activation='relu'),
#     layers.Dropout(0.2),
#     layers.Dense(1, activation='sigmoid'),
# ])


In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 64)                2097216   
                                                                 
 dense_1 (Dense)             (None, 16)                1040      
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 25,685,985
Trainable params: 25,632,865
Non-trainable params: 53,120
_________________________________________________________________


In [23]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(epsilon=0.01),
    loss = 'binary_crossentropy',
    metrics = ['accuracy'],
)

## Step 3 : Train Model 

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=50,
)

Epoch 1/50
 83/149 [===============>..............] - ETA: 14:51 - loss: 0.1344 - accuracy: 0.9426

In [ ]:
import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();

## Conclusion